### **Inference File**

In [1]:
!git clone https://github.com/pranjali97/Relation-Extraction-using-multi-Label-classification-.git

fatal: destination path 'Relation-Extraction-using-multi-Label-classification-' already exists and is not an empty directory.


In [2]:
import pandas as pd
import numpy as np
from pickle import load
import torch
import scipy

### Load Model Artifacts

In [3]:
# load the model
model = torch.load(open('Iteration-10.pkl', 'rb'))
# load the binarizer
binarizer = load(open('binarizer.pkl', 'rb'))
#load tfidf vectorizer
vectorizer = load(open('vectorizer.pk', 'rb'))

In [4]:
import pandas as pd
import numpy as np

test_data = pd.read_csv('Relation-Extraction-using-multi-Label-classification-/hw1_test-2.csv')
print('Test data points: ', test_data.shape)

Test data points:  (981, 2)


In [5]:
test_data

,ID,UTTERANCES
0,0,star of thor
1,1,who is in the movie the campaign
2,2,list the cast of the movie the campaign
3,3,who was in twilight
4,4,who is in vulguria
...,...,...
976,976,trailer for star wars a new hope
977,977,show resident evil movies with trailers
978,978,can i see previews for upcoming warner brother...
979,979,how many woody allen movies are set in new yor...


### Process Text

In [6]:
# text processing
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
nltk.download('omw-1.4')
nltk.download('wordnet')


w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, 'v') for w in w_tokenizer.tokenize(text)])


# strip extra spaces
test_data['processed_text'] = test_data.UTTERANCES.str.strip()
#lower case data
test_data['processed_text'] = test_data.processed_text.str.lower()

# Remove stopwords
test_data['processed_text'] = test_data['processed_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_data['processed_text'] = test_data.processed_text.apply(lemmatize_text)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
print(test_data)

      ID                                         UTTERANCES  \
0      0                                       star of thor   
1      1                   who is in the movie the campaign   
2      2            list the cast of the movie the campaign   
3      3                                who was in twilight   
4      4                                 who is in vulguria   
..   ...                                                ...   
976  976                   trailer for star wars a new hope   
977  977            show resident evil movies with trailers   
978  978  can i see previews for upcoming warner brother...   
979  979  how many woody allen movies are set in new yor...   
980  980      how many scorsese films were filmed in france   

                                  processed_text  
0                                      star thor  
1                                 movie campaign  
2                       list cast movie campaign  
3                                      

### Encode Data

In [8]:
# vectorizer utterances

x_test = vectorizer.transform(test_data['processed_text'])
x_test = torch.tensor(scipy.sparse.csr_matrix.todense(x_test)).float()

In [9]:
x_test.shape

torch.Size([981, 889])

### Infer data

In [10]:
# load model
threshold = 0.14
with torch.no_grad():
    model.eval()
    log_ps = model(x_test)
    pred = torch.sigmoid(log_ps)
    pred = (pred>threshold).int().tolist()
    


In [11]:
labels_batch = []
for i in range(len(pred)):
  label = binarizer.inverse_transform(np.array([pred[i]]))
  labels_batch.append(label)

In [12]:
processed_labels = []
for row in labels_batch:
  label_list = ([' '.join(tups) for tups in row])
  label_text = label_list[0].replace('none', '')
  label_text = label_text.strip()
  processed_labels.append(label_text)
  # print(label_list[0].replace(''))
  # labels = lable

In [13]:
test_data['CORE RELATIONS'] = processed_labels

In [14]:
test_data

,ID,UTTERANCES,processed_text,CORE RELATIONS
0,0,star of thor,star thor,movie.starring.actor
1,1,who is in the movie the campaign,movie campaign,
2,2,list the cast of the movie the campaign,list cast movie campaign,movie.starring.actor
3,3,who was in twilight,twilight,movie.starring.actor
4,4,who is in vulguria,vulguria,
...,...,...,...,...
976,976,trailer for star wars a new hope,trailer star war new hope,
977,977,show resident evil movies with trailers,show resident evil movies trailers,
978,978,can i see previews for upcoming warner brother...,see preview upcoming warner brothers movies,movie.production_companies
979,979,how many woody allen movies are set in new yor...,many woody allen movies set new york city,movie.directed_by movie.initial_release_date m...


In [15]:
submission_df = test_data[['ID', 'CORE RELATIONS']]

In [16]:
submission_df.to_csv('submission.csv',index=False)